In [1]:
import os
os.environ['HTTP_PROXY'] = "127.0.0.1:10809"
os.environ['HTTPS_PROXY']="127.0.0.1:10809"
os.environ["OPENAI_API_KEY"] = ""

In [2]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name='gpt-3.5-turbo-16k', temperature=0.1)

## 从key文件读取key值
* 输入参数: file_path 文件路径
* 返回值: 存有key的列表

In [3]:
def read_keys_from_file(file_path):
    key_pool = []
    with open(file_path, 'r') as file:
        for line in file:
            key = line.strip()  # 去掉每一行的换行符
            key_pool.append(key)  # 将每一行存入key池中
    return key_pool

In [4]:
# 读取key
file_path = "key.txt"  # 将这个替换为你的txt文件的路径
key_pool = read_keys_from_file(file_path)

## 读取PDF文档函数
* 参数:文件路径file_path,提取前n页
* 返回值:一个列表

In [5]:
import pdfplumber

def read_pdf(file_path, n):
    inner = []
    text = ""
    with pdfplumber.open(file_path) as pdf:
            # 提取前n页
            for i, page in enumerate(pdf.pages):
                # 如果已经处理了n页，就停止
                if i == n:
                    break
                inner.append(page.extract_text())
    return inner

# 按页数进行划分
* 参数:inner->一个装有每一页pdf的列表
* 参数:group_size->每个组的大小
* 返回值:一个列表

In [6]:
def group_inner(inner, group_size):
    # 将inner按照每n页为一组进行分割
    grouped_inner = [inner[i:i + group_size] for i in range(0, len(inner), group_size)]
    return grouped_inner

In [7]:
content = read_pdf(r'./data/KT820说明书V2.0 201909.pdf', 15)

In [8]:
inner = group_inner(content, 5)

In [9]:
print(len(inner))

3


In [10]:
from langchain.prompts import  ChatPromptTemplate
prompts = """
    ----
    {text}
    ----
    帮我把上面内容总结成2-4个目录，目录不要子目录，目录为简洁摘要，其它内容不用给我
    """

prompts2 = """
    abstract:
    ----
    {abstract}
    ----
    text:
    ----
    {text}
    ----

    跟据上面的摘要，找出下面内容中每个摘要对应的开始位置内容：
    返回格式:
    {respond_struction}

"""

prompts3 = """
    text:
    ----
    {text}
    ----

    Json
    '''
    {Json}
    '''
    根据上面的json的内容,从text中找到对应的段落并且返回符合逻辑的内容回来,text用---分割,json用'''来分割

    返回格式:
    {respond_struction}

"""
promptsTemplate = ChatPromptTemplate.from_template(prompts)
promptsTemplate2 = ChatPromptTemplate.from_template(prompts2)
promptsTemplate3 = ChatPromptTemplate.from_template(prompts3)

## 指定返回值格式
> 返回值格式采用json数据,包含三部分:关键词,开始词,结束词
### langchain的json格式
1. 先定义单个模式,Schema = ResponseSchema(name="",description="")
2. 然后将多个模式组合在一起response_schemas = [abstract_schema, start_schema, end_schema]
3. 通过结构输出解析函数解析成结构对象 = StructuredOutputParser(response_schemas = response_schemas)
4. 最后将结构对象解析成json格式的内容


In [11]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
abstract_schema = ResponseSchema(name="abstract",description="这是摘要部分的内容, 也就是用来记录摘要的")
content_schema = ResponseSchema(name="content",description="返回和上面摘要部分相关的全部内容,有n个摘要就返回n个内容")

response_schemas = [abstract_schema, content_schema]
out_parse = StructuredOutputParser(response_schemas = response_schemas)
format_instructions = out_parse.get_format_instructions()

## 制作两条链来定位
* 第一条链用来摘要总结文本内容,第一条链用3.5的模型
* 第二条链用来根据摘要来定位文本内容,第二条链用3.5模型

In [12]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=promptsTemplate)
# llm2 = ChatOpenAI(model_name='gpt-4-0613', temperature=0.2)
chain2 = LLMChain(llm=llm, prompt=promptsTemplate2)

In [ ]:
import random
abstract=[]
result = []
for text in inner:
    os.environ["OPENAI_API_KEY"] = key_pool[random.randint(0,len(key_pool)-1)]
    print(os.environ.get("OPENAI_API_KEY"))
    respones=chain.run(text = text)
    res = chain2.run(text=text,abstract=respones,respond_struction=format_instructions)
    abstract.append(respones)
    result.append(res)

In [17]:
print(result[2])

```json
{
	"abstract": "1. 编程基础",
	"content": "第一篇\n编程篇\n9"
}
{
	"abstract": "2. 主要技术指标",
	"content": "功 能 描 述 规 格 指 标\n控制轴数 3轴 （X、Z、C轴）\n控制轴\n联动轴数 2轴\n最小设定单位 X：0.001mm Z：0.001mm\n输入指令 最小移动单位 0.001mm\n最大指令值 ±99999.999mm\n最大移动速度 60000mm/min\n螺纹导程 0.001mm～500.000mm\n进给 自动加减速 直线，前加减速\n进给速度倍率 0～150%\n快速速度倍率 Fo～100%,Fo由参数设定\n手动连续进给 X,Z；手动进给速度按键设定\n三种回零：方式B（Z脉冲中断方式），方式 C\n返回机床零点\n（回零定位开关），方式A（浮动零点）\n手动 返回程序零点 快速回加工起始点\n单步增量进给 进给当量0.001mm,0.01mm,0.1mm，1mm\n倍率：x1,x10,x100；轴选：X，Z，C；按键或\n手轮进给\n外部输入口选控倍率和轴选\n直线、圆弧、螺纹循环、攻丝循环、钻孔循环、\n插补 定位，插补功能\n外圆、端面复合循环等功能\n程序存储容量 电子盘：32M字节\n存储程序个数 480个\n存储及编辑\n程序编辑 插入，修改，删除，复制\n参数存储 参数备份，恢复出厂值，参数U盘导入导出\n8"
}
{
	"abstract": "3. 系统功能",
	"content": "功 能 描 述 规 格 指 标\n液晶显示 8英寸，TFT真彩显示\n位置，程序，刀补，\n显示\n报警，诊断，参数， 显示内容丰富，直观\n设置，U盘,图形\n程序导入导出 有\n第\nU盘功能 参数导入导出 有 一\n篇\n系统U盘升级 有\n编\n输入口 54路开关量，光电隔离输入 程\n篇\n输出口 48路开关量输出（OC输出）\n变频器模拟量控制或 S1～S4 档位控制；主轴\n主轴功能\n模拟量输出倍率可调0～150%；\nM，S，T机能\n刀位号：T01～T08，刀补号：01～24；电动刀\n刀具功能 架，排刀刀架或专用刀架；运行中修整刀补值；\n程序控制动态刀补补偿。\n辅助T功能 有，特定T代码执行特定子程序\n辅助M

In [11]:
print(result)
print(len(result))
print(len(inner[2]))

```json
{
	"abstract": "G31指令：用于检测输入口信号并控制程序跳转",
	"content": "若在 Z 轴走完 W160 后仍未检测到 8 号输入口的低电平信号，系统结束 G31 段，执行\nG0 U60段。\nZ\n160\nA C B\n实际移动\n第\n一 无skip信号时的移动\n06\n篇\n编\nD D'\n程 skip信号输入点\nX\n篇\n图2-4\n举例2：如下图2－5，轨迹A－B－D为无跳转信号的运行轨迹\n执行 G31 W80 K6 F200\nG01 X300 Z100\n程序执行时，以F200的速度进给Z轴，同时检测6号输入口状态，在走到C点位置时，\n系统检测到6号输入口的高电平信号，程序结束G31段执行，立刻跳转到G01 X300 Z100段\n执行。这样，实际运行轨迹为 A－C－D。\n若Z轴到达B点后仍未检测到6号输入口的高电平信号，系统结束G31段，执行G01 X300\nZ100段。\nZ\nA(100,0) C B(100,80)\nD(300,100)\nskip信号输入点\nX\n图2－5"
}
{
	"abstract": "G32指令：用于切削直螺纹和锥螺纹",
	"content": "用G32指令，可以切削导程不变的直螺纹，锥螺纹。\n直螺纹指令格式：G32 Z(W)__F/I__；\nZ（W）：螺纹终点 Z向位置；\nF：公制螺纹，长轴方向的导程（0.001—500.000mm）。\nI：英制螺纹，长轴方向的每英寸牙数（1—25400牙/英寸）\n锥螺纹指令格式：G32 X（U）__Z(W)__F/I__；\nX（U）：螺纹终点 X向位置；\nZ（W）：螺纹终点 Z向位置；\n22\nF：公制螺纹，长轴方向的导程（0.001—500.000mm）。\nI：英制螺纹，长轴方向的每英寸牙数（1—25400牙/英寸）\n在螺纹切削开始及结束部分，一般由于升降速的原因，会出现导程不正确部分，考虑此\n因素影响，指令螺纹长度应当比需要的螺纹长度要长些。\n例1：直螺纹切削\n68 第\n一\n篇\nZ\n编\n2 1 程\n篇\n52\nX\n在Z方向：△1=3mm，△2=1.5mm\n螺纹导程：3mm\n在牙深共2mm（公制输入，直径编程）：\nG00 U-25.0 //定位，进刀深度为2

1. [{},{},{}]
2. 输出文件